In [1]:
!pwd

/root


In [2]:
import os

In [3]:
os.chdir('/root/autodl-tmp/generativeModel/disguish_learning/LR_001')

In [4]:
import transformers
from transformers import  AutoModelForSeq2SeqLM,AutoTokenizer

In [5]:


import torch
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from datasets import load_dataset


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [6]:
device = torch.device('cuda:0')

In [7]:
cache_dir = '/root/autodl-tmp/model/'
model = AutoModelForSeq2SeqLM.from_pretrained('THUDM/glm-2b',cache_dir=cache_dir,
                                              trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of the model checkpoint at THUDM/glm-2b were not used when initializing GLMForConditionalGeneration: ['out_proj.bias', 'dense.bias', 'dense.weight', 'out_proj.weight']
- This IS expected if you are initializing GLMForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GLMForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
model = prepare_model_for_int8_training(model)
lora_config = LoraConfig(
    r=8,#LORA_R,
    lora_alpha=16,#LORA_ALPHA,
    target_modules=["query_key_value"],#TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model,lora_config).to(device)

/root/miniconda3/lib/python3.8/site-packages/peft/tuners/lora.py:180: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [9]:
data_cache_dir = '/root/autodl-tmp/data/'

In [10]:
data = load_dataset("cahya/instructions-zh",cache_dir=data_cache_dir)

Using custom data configuration cahya--instructions-zh-507237297bfcf9f5
Found cached dataset parquet (/root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
for x in data['train']:
    print(x)
    break

{'id': 42216, 'text': 'User: 虚假信息与虚假信息有什么区别?\nAssistant: 误导和误导之间有区别 - 误导是欺骗的使用来破坏沟通过程,而误导是错误的信息或事实的误解。 误导的一个常见例子是新闻来源故意报告虚假声明或误导性信息以传播宣传和影响意见。'}


In [12]:
tokenizer = AutoTokenizer.from_pretrained('THUDM/glm-2b',cache_dir=cache_dir,trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [13]:
def split_train_example(text:str):
    answer_prefix = "Assistant:"
    prompt_prefix = "User:"

    answer_start_idx = text.find(answer_prefix)
    if(answer_start_idx > 0):
        # this is an trian data
        answer = text[answer_start_idx + len(answer_prefix):]
        prompt = text[:answer_start_idx].replace(prompt_prefix,"")
    else:
        prompt = text
        answer = None

    return prompt,answer

In [14]:
def build_tokenzie_func(tokenizer,pad_idx=0,max_length=256,pad=True):
    def tokenize(example):
        text = example['text']
        prompt,answer = split_train_example(text)
        prompt_idxs = tokenizer(prompt)
        answer_idxs = tokenizer(answer) if answer is not None else []

        example = {}
        example['input_ids'] = prompt_idxs['input_ids'] + answer_idxs['input_ids']
        example['attention_mask'] = prompt_idxs['attention_mask'] + answer_idxs['attention_mask']
        example['labels'] = [0] * len(prompt_idxs['attention_mask']) + answer_idxs['input_ids']
        example['prompt'] = prompt
        example['answer'] = answer
        
        if(len(example['input_ids']) < max_length):
            count = max_length - len(example['input_ids'])
            example['input_ids'] = example['input_ids'] + [0] * count
            example['attention_mask'] = example['attention_mask']+ [0] * count
            example['labels'] = example['labels']+ [0] * count
        
        example['input_ids'] = example['input_ids'][:max_length]
        example['attention_mask'] = example['attention_mask'][:max_length]
        example['labels'] = example['labels'][:max_length]
        

        return example
    return tokenize
tokenize_func = build_tokenzie_func(tokenizer)

In [15]:
train_data = data['train'].map(tokenize_func)

Loading cached processed dataset at /root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-9204c8ca728fc55b.arrow


In [16]:
print(train_data[0])

{'id': 42216, 'text': 'User: 虚假信息与虚假信息有什么区别?\nAssistant: 误导和误导之间有区别 - 误导是欺骗的使用来破坏沟通过程,而误导是错误的信息或事实的误解。 误导的一个常见例子是新闻来源故意报告虚假声明或误导性信息以传播宣传和影响意见。', 'input_ids': [50259, 5525, 247, 248, 161, 223, 229, 46479, 94, 162, 223, 107, 10310, 236, 164, 247, 248, 161, 223, 229, 46479, 94, 162, 223, 107, 17312, 231, 20015, 222, 20046, 230, 44293, 118, 26344, 104, 30, 198, 50256, 50259, 5525, 107, 107, 43380, 120, 161, 240, 234, 46237, 107, 43380, 120, 45298, 29785, 112, 17312, 231, 44293, 118, 26344, 104, 532, 5525, 107, 107, 43380, 120, 42468, 162, 105, 118, 165, 103, 245, 21410, 45635, 18796, 101, 30266, 98, 163, 254, 112, 161, 251, 237, 162, 110, 253, 34460, 248, 32573, 229, 163, 101, 233, 11, 32003, 234, 46237, 107, 43380, 120, 42468, 165, 242, 247, 46237, 107, 21410, 46479, 94, 162, 223, 107, 22755, 244, 12859, 233, 22522, 252, 21410, 46237, 107, 164, 100, 96, 16764, 5525, 107, 107, 43380, 120, 21410, 31660, 10310, 103, 30585, 116, 164, 100, 223, 160, 122, 233, 36310, 42468, 23877, 108, 29785, 1

In [17]:
val_data = data['validation'].map(tokenize_func)
test_data = data['test'].map(tokenize_func)

Loading cached processed dataset at /root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e9d2e850985c6bf9.arrow
Loading cached processed dataset at /root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c7f28477d773e7d7.arrow


In [18]:
ddp = True if torch.cuda.device_count() > 1 else False
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=128 // 8,
        warmup_steps=100,
        num_train_epochs=1,
        learning_rate=1e-5,
        fp16=True,
        logging_steps=20,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        save_steps=200,
        output_dir="lora-alpaca",
        save_total_limit=3,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))


In [ ]:
if torch.__version__ >= "2":
    model = torch.compile(model)

##############
# 训练循环

trainer.train()


model.save_pretrained("lora-alpaca")

print("\n If there's a warning about missing keys above, please disregard :)")

In [22]:
prompt = "Assistant:你好，我的第一个助理"

In [1]:
prompt_idxs = tokenizer(prompt)
prompt_idxs = {
    key : torch.LongTensor(prompt_idxs[key])
    for key in prompt_idxs
}


NameError: name 'tokenizer' is not defined

In [28]:
prompt_idxs

{'input_ids': tensor([5.0259e+04, 4.8902e+04, 2.5000e+01, 1.9526e+04, 2.5400e+02, 2.5001e+04,
         1.2100e+02, 1.7100e+02, 1.2000e+02, 2.3400e+02, 2.2755e+04, 2.3900e+02,
         2.1410e+04, 1.6300e+02, 1.0500e+02, 1.0500e+02, 3.1660e+04, 1.0310e+04,
         1.0300e+02, 2.7950e+04, 1.0200e+02, 4.9426e+04, 2.2800e+02, 5.0256e+04]),
 'attention_mask': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1.])}

In [30]:
torch._dynamo.config.suppress_errors = True
model(input_ids=prompt_idxs['input_ids'],attention_mask=prompt_idxs['attention_mask'])

[2023-04-23 15:57:14,107] torch._dynamo.convert_frame: [ERROR] WON'T CONVERT forward /root/.cache/huggingface/modules/transformers_modules/THUDM/glm-2b/774fda883d7ad028b8effc3c65afec510fce9634/modeling_glm.py line 184 
due to: 
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/torch/_subclasses/fake_tensor.py", line 920, in merge_devices
    raise RuntimeError(
RuntimeError: Unhandled FakeTensor Device Propagation for aten.index_select.default, found two different devices cuda:0, cpu

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/torch/_dynamo/utils.py", line 1206, in run_node
    raise RuntimeError(
RuntimeError: Failed running call_function <function embedding at 0x7fd4d6734af0>(*(FakeTensor(FakeTensor(..., device='meta', size=(24,)), cpu), FakeTensor(Parameter(FakeTensor(..., device='meta', size=(50304, 2048))), cuda:0), None, None, 2.

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)